In [1]:
# Install torch geometric
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch_geometric
torch_geometric.__version__

'2.3.1'

In [3]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class GNNStack(torch.nn.Module):                                                   #this makes a subclass of the nn.Module 
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):        #instance of the subclass(constructor method)
        super(GNNStack, self).__init__()                                           # initiates GNNstack instance 
        conv_model = self.build_conv_model(args.model_type)                        #building the conv layers (3 for us)         
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))


        self.post_mp = nn.Sequential(                                                  #post processing dropout
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout),
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT
        
    def forward(self,data):
        x, pos_edges, neg_edges= data.x, data.edge_index , data.neg_edges
        for i in range(self.num_layers):
            x = self.convs[i](x,pos_edges,neg_edges)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout,training=self.training)
            
        x=self.post_mp(x)
        
        if self.emb==True:
            return x
        return F.log_softmax(x, dim=1)
    
    def get_node_embeddings(self, x):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=False)  # Set training to False to get deterministic output
        return x
    
    def loss(self, pred, label):
        return F.binary_cross_entropy_with_logits(pred, label) # binary cross-entropy loss

In [4]:
class GraphSage(MessagePassing):
    
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(GraphSage, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        self.lin_l = None
        self.lin_r = None

        ############################################################################
        # TODO: Your code here! 
        # Define the layers needed for the message and update functions below.
        # self.lin_l is the linear transformation that you apply to embedding 
        #            for central node.
        # self.lin_r is the linear transformation that you apply to aggregated 
        #            message from neighbors.
        # Don't forget the bias!
        # Our implementation is ~2 lines, but don't worry if you deviate from this.
        self.lin_l = nn.Linear(self.in_channels, self.out_channels, bias=True)
        self.lin_r = nn.Linear(self.in_channels, self.out_channels, bias=True)
        ############################################################################

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x,edge_index,neg_edges,size = None):
        out = None

        ############################################################################
        # TODO: Your code here! 
        # Implement message passing, as well as any post-processing (our update rule).
        # 1. Call the propagate function to conduct the message passing.
        #    1.1 See the description of propagate above or the following link for more information: 
        #        https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html
        #    1.2 We will only use the representation for neighbor nodes (x_j), so by default
        #        we pass the same representation for central and neighbor nodes as x=(x, x). 
        # 2. Update our node embedding with skip connection from the previous layer.
        # 3. If normalize is set, do L-2 normalization (defined in 
        #    torch.nn.functional)
        #
        # Our implementation is ~5 lines, but don't worry if you deviate from this.
        
        pos=self.propagate(pos_edges, x=(x, x), size=size)
        neg=self.propagate(neg_edges, x=(x, x), size=size)
        out=self.lin_l(x) + self.lin_r(pos) - self.lin_r(neg)
        if self.normalize:
            out = F.normalize(out, p=2)
        ############################################################################

        return out

    def message(self, x_j):

        out = None

        ############################################################################
        # TODO: Your code here! 
        # Implement your message function here.
        # Hint: Look at the formulation of the mean aggregation function, focusing on 
        # what message each neighboring node passes.
        #
        # Our implementation is ~1 lines, but don't worry if you deviate from this.
        out = x_j
        ############################################################################

        return out

    def aggregate(self, inputs, index, dim_size = None):

        out = None

        # The axis along which to index number of nodes.
        node_dim = self.node_dim

        ############################################################################
        # TODO: Your code here! 
        # Implement your aggregate function here.
        # See here as how to use torch_scatter.scatter: 
        # https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html#torch_scatter.scatter
        #
        # Our implementation is ~1 lines, but don't worry if you deviate from this.

        out = torch_scatter.scatter(inputs, index, node_dim, dim_size=dim_size, reduce='mean')
        ############################################################################

        return out


In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
from torch_geometric.data import Data
import scipy.sparse
from torch import Tensor
from torch.utils.dlpack import from_dlpack, to_dlpack

from torch_geometric.utils.num_nodes import maybe_num_nodes

In [6]:
pos_edges=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/train.csv')

In [7]:
pos_edges = torch.tensor(pos_edges.values).T
num_nodes = pos_edges.max().item()
print(num_nodes)

1862220


In [8]:
def to_scipy_sparse_matrix(
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    num_nodes: Optional[int] = None,
) -> scipy.sparse.coo_matrix:
    row, col = edge_index.cpu()

    if edge_attr is None:
        edge_attr = torch.ones(row.size(0))
    else:
        edge_attr = edge_attr.view(-1).cpu()
        assert edge_attr.size(0) == row.size(0)

    N = maybe_num_nodes(edge_index, num_nodes)
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (N, N))
    return out

In [9]:
adj_mat=to_scipy_sparse_matrix(pos_edges)

In [10]:
from torch_geometric.utils import negative_sampling
x = torch.rand(num_nodes+1, 16)
neg_edges=negative_sampling(pos_edges)

In [11]:
FB = Data(x=x, edge_index=pos_edges)
FB.neg_edges=neg_edges

In [12]:
num_nodes = FB.x.shape[0]
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
FB.train_mask = train_mask

In [13]:
FB

Data(x=[1862221, 16], edge_index=[2, 9437519], neg_edges=[2, 9437519], train_mask=[1862221])

In [14]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [15]:
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args,batch_size):

    print("Node feature task")
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim,2, 
                            args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        model.train()
        pos_loader = DataLoader(pos_edges, batch_size=batch_size, shuffle=True)
        neg_loader = DataLoader(neg_edges, batch_size=batch_size, shuffle=True)
        emb = model(dataset)
        total_loss = 0 
        for pos_batch, neg_batch in zip(pos_loader, neg_loader):
            pos_batch = pos_batch[train_mask[pos_batch[:, 0]] & train_mask[pos_batch[:, 1]]]
            neg_batch = neg_batch[train_mask[neg_batch[:, 0]] & train_mask[neg_batch[:, 1]]]
            if len(pos_batch) == 0 or len(neg_batch) == 0:
                continue
            
            pos_dot_product = torch.bmm(emb[pos_batch[:, 0]].unsqueeze(2), emb[pos_batch[:, 1]].unsqueeze(2)).squeeze()
            neg_dot_product = torch.bmm(emb[neg_batch[:, 0]].unsqueeze(2), emb[neg_batch[:, 1]].unsqueeze(2)).squeeze()
            pos_edge_scores = torch.sigmoid(pos_dot_product)
            neg_edge_scores = torch.sigmoid(neg_dot_product)
            pos_labels = torch.ones(pos_edge_scores.shape)  # Label 1 for positive edges
            neg_labels = torch.zeros(neg_edge_scores.shape)  # Label 0 for negative edges

            pos_loss = model.loss(pos_edge_scores, pos_labels)
            neg_loss = model.loss(neg_edge_scores, neg_labels)
            loss = pos_loss + neg_loss
            loss.backward()
            opt.step()
            total_loss += loss.item()
        
        losses.append(total_loss)
        print(f"Epoch: {epoch+1}, Loss: {total_loss}, Embeddings: {emb}")
        
    return emb

In [16]:
class objectview:
    def __init__(self, d):
        self.__dict__ = d

In [17]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    for args in [
        {'model_type': 'GraphSage', 'dataset': 'FB', 'num_layers': 2, 'heads': 1, 'batch_size': 1000, 'hidden_dim': 32, 'dropout': 0.5, 'epochs': 500, 'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0, 'weight_decay': 5e-3, 'lr': 0.01},
    ]:
        args = objectview(args)
        for model in ['GraphSage']:
            args.model_type = model

            if args.dataset == 'FB':
                dataset = FB
            else:
                raise NotImplementedError("Unknown dataset")
            NODE_EMB = train(dataset, args,1000)

/Users/akshaj.g/mamba/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Node feature task



Training:   0%|                             | 1/500 [00:03<29:52,  3.59s/Epochs]

Epoch: 1, Loss: 0, Embeddings: tensor([[-0.5936, -0.8037],
        [-0.4308, -1.0497],
        [-0.6197, -0.7724],
        ...,
        [-0.6466, -0.7420],
        [-0.6126, -0.7808],
        [-0.5972, -0.7992]], grad_fn=<LogSoftmaxBackward0>)


Training:   0%|                             | 2/500 [00:07<33:46,  4.07s/Epochs]

...

Epoch: 4, Loss: 0, Embeddings: tensor([[-0.5576, -0.8500],
        [-0.5619, -0.8443],
        [-0.5759, -0.8260],
        ...,
        [-0.5130, -0.9131],
        [-0.6450, -0.7437],
        [-0.5891, -0.8092]], grad_fn=<LogSoftmaxBackward0>)


Training:   1%|▎                            | 5/500 [00:19<32:03,  3.89s/Epochs]

Epoch: 5, Loss: 0, Embeddings: tensor([[-0.5207, -0.9017],
        [-0.5150, -0.9101],
        [-0.6506, -0.7376],
        ...,
        [-0.6059, -0.7887],
        [-0.5156, -0.9092],
        [-0.5733, -0.8293]], grad_fn=<LogSoftmaxBackward0>)


Training:   1%|▎                            | 5/500 [00:22<37:02,  4.49s/Epochs]


KeyboardInterrupt: 

In [ ]:
'''import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args):

    print("Node feature task")
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim,2, 
                            args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        for batch in loader:
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
          test_acc = test(test_loader, model)
          test_accs.append(test_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])
    
    return test_accs, losses, best_model, best_acc, test_loader'''

In [ ]:
FB